# 라이브러리 임포트

In [15]:
import time
from os import write

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
import pymysql
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 로드, 영화 이름 설정

In [146]:
movie_name= ['파묘','범죄도시4','베테랑2','파일럿','웡카','인사이드아웃2','쿵푸팬더4','슈퍼배드4','위시','사랑의하츄핑']
animation_name = ['인사이드아웃2','쿵푸팬더4','슈퍼배드4','위시','사랑의하츄핑']


service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

In [147]:
i=2
# 영화 '베테랑'
driver.get('https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EC%98%81%ED%99%94+'+movie_name[i])

# 개봉일 부분 찾기
raw_released_date = driver.find_elements(By.XPATH,'/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div[2]/div/div[1]/dl/div[2]/dd')

released_date = raw_released_date[0].text
print(released_date)

2024.09.13.


In [148]:
#관람평 창으로 이동
driver.get('https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&ssc=tab.nx.all&query=%EC%98%81%ED%99%94+'movie_name[i]'+%EA%B4%80%EB%9E%8C%ED%8F%89')
time.sleep(0.5)


In [149]:
# 영화 제목
raw_movie_title = driver.find_elements(By.XPATH, '/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[1]/div[1]/h2/span/strong')
movie_title = raw_movie_title[0].text
print(movie_title)
time.sleep(0.5)


베테랑2


In [150]:
#mysql server 연결

import pymysql
# DB 접속 정보 설정
connection = pymysql.connect(
    host='127.0.0.1',       # 로컬이면 127.0.0."1 또는 localhost
    user='root',            # MySQL 접속 사용자명
    password='Enter your password',    # MySQL 사용자 비번 (없다면 '' 빈문자)
    db='movie',        # 사용할 DB명
    charset='utf8mb4'       # 인코딩(utf8mb4 권장)
)

In [151]:
# 드라이버에서 웹페이지 스크롤 내리기 (y축 방향으로 +600)
ActionChains(driver) \
        .scroll_by_amount(0, 600) \
        .perform()
# 아래 코드 진행을 위해 잠시 딜레이시키기
time.sleep(1)

# 크롤링 시작

In [152]:
# 영화 리뷰에 대한 인덱스
comment_index = 1
# 리뷰가 공백인 경우 카운트
nonsave_number = 0
# pymysql 연결
mydb = pymysql.connect(host='localhost', user='root', password='gksmf3191@', db='movie', charset='utf8')
mycursor = mydb.cursor()
# insert sql query
sql = "INSERT INTO reviewInformation(movie_title, comment, star_score, writing_date, released_date) VALUES (%s, %s, %s, %s, %s)"
# 영화 제목, 개봉일 출력
print(movie_title)
print(released_date)
time.sleep(1)

베테랑2
2024.09.13.


In [153]:
scroll_origin = ScrollOrigin.from_viewport(140, 641) #드라이버 내에서, 영화 관람평 창에서 스크롤을 진행하기 위해 원점 설정

In [154]:
# 크롤링 및 DB 저장
comment_index = 1 
crawl_rounds = 20 #5개씩 크롤링 진행할 때, 몇 번의 라운드를 할 것인지

for round_idx in range(crawl_rounds):
    print("\n=== [라운드 " + str(round_idx + 1) + "/" + str(crawl_rounds) + "] 리뷰 5개 크롤링 시작 ===\n")
    
    # 이번 라운드에서 리뷰가 더 이상 없어 break 되었는지 여부
    no_more_reviews = False

    for i in range(5):
        # 1) 관람객 코멘트 부분
        try:
            xpath_comment = "/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[4]/div[4]/ul/li[" + str(comment_index) + "]/div[2]/div/span[2]"
            raw_comment = driver.find_elements(By.XPATH, xpath_comment)
        except Exception as e:
            print(f"Selector 예외 발생(코멘트 부분): {e}")
            no_more_reviews = True
            break

        if len(raw_comment) == 0:
            print("[종료] li[" + str(comment_index) + "] (코멘트) 찾지 못함 → 더 이상 리뷰가 없다고 판단.")
            no_more_reviews = True
            break
        comment = raw_comment[0].text

        # 2) 관람객 별점 부분
        try:
            xpath_star = "/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[4]/div[4]/ul/li[" + str(comment_index) + "]/div[1]/div/div[2]"
            raw_star_score = driver.find_elements(By.XPATH, xpath_star)
            star_score = raw_star_score[0].text if len(raw_star_score) > 0 else ""
        except Exception as e:
            print(f"Selector 예외 발생(별점 부분): {e}")
            star_score = ""

        # 3) 리뷰 작성일 부분
        try:
            xpath_date = "/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[4]/div[4]/ul/li[" + str(comment_index) + "]/dl/dd[2]"
            raw_writing_date = driver.find_elements(By.XPATH, xpath_date)
            writing_date = raw_writing_date[0].text if len(raw_writing_date) > 0 else ""
        except Exception as e:
            print(f"Selector 예외 발생(작성일 부분): {e}")
            writing_date = ""

        # DB 저장
        mycursor.execute(sql, (movie_title, comment, star_score, writing_date, released_date))
        mydb.commit()

        # 콘솔 출력
        print("[리뷰 #" + str(comment_index) + "] 작성일: " + writing_date + ", 별점: " + star_score)
        print("감상평:", comment)

        comment_index += 1
        time.sleep(0.5)  # 리뷰 하나 가져올 때마다 조금 대기 (과도한 요청 방지)
    
    # 5개 반복(for i in range(5)) 끝
    if no_more_reviews:
        print("\n[더 이상 리뷰가 없어 라운드 중단]\n")
        break
    
    # 라운드(5개) 끝난 후, 스크롤 800px
    print("\n=== [라운드 " + str(round_idx + 1) + "] 종료: 800px 스크롤 내립니다. ===\n")
    ActionChains(driver).scroll_from_origin(scroll_origin, 0, 800).perform()
    time.sleep(3)  # 스크롤 후 로딩 대기

# --------------------------------------------------------
# 4) 마무리 (드라이버 종료 & DB 커넥션 정리)
# --------------------------------------------------------
print("\n[모든 크롤링 작업 종료] 드라이버 및 DB 연결을 닫습니다.")
driver.quit()
mycursor.close()
mydb.close()


=== [라운드 1/20] 리뷰 5개 크롤링 시작 ===

[리뷰 #1] 작성일: 2024.09.13. 13:15, 별점: 별점(10점 만점 중)
6
감상평: 영화 보고 나온 지금 내 기분이 그래 어이가 없네.
[리뷰 #2] 작성일: 2024.09.13. 14:37, 별점: 별점(10점 만점 중)
5
감상평: 류승완감독한테 감 한상자 보내드려야겠네요감좀 찾으시라고
[리뷰 #3] 작성일: 2024.09.13. 15:24, 별점: 별점(10점 만점 중)
2
감상평: 9년 동안 여기저기 상한 재료만 모아서 고추장 없이 비빈 비빔밥
[리뷰 #4] 작성일: 2024.09.13. 13:25, 별점: 별점(10점 만점 중)
6
감상평: 베테랑2 아니고 비질란테 극장판
[리뷰 #5] 작성일: 2024.09.13. 18:44, 별점: 별점(10점 만점 중)
1
감상평: 이걸 칸에서 틀었다고요?..

=== [라운드 1] 종료: 800px 스크롤 내립니다. ===


=== [라운드 2/20] 리뷰 5개 크롤링 시작 ===

[리뷰 #6] 작성일: 2024.09.13. 11:42, 별점: 별점(10점 만점 중)
2
감상평: 모든 면에서 허술한 영화
[리뷰 #7] 작성일: 2024.09.13. 13:15, 별점: 별점(10점 만점 중)
1
감상평: 인방을 영화로 좀 만들지 말라고.. 제대로 만드는 걸 못봤음. 스토리도 난잡하고 이게 뭔가 싶다
[리뷰 #8] 작성일: 2024.09.13. 11:17, 별점: 별점(10점 만점 중)
5
감상평: 왜 그래야만했는지흑화되는 과정없는 단순히 그럴려고 했다는이해되지않는 빌런에통쾌함도 없는 가해자 죽이기허무하고 허술하고 재미도 없는 막판씬해외영화제 왜 초청된겨?류감독은 좀 더 스토리를 연구해얄듯..쿠키처럼..장난하냐? 베테랑 1때만한 즐거움 재미가 절반으로 뚝
[리뷰 #9] 작성일: 2024.09.13. 13:45, 별점: 별점(10점 만점 중)
6
감상평: 그 빌런인지 악연?.. 왜 그렇게 되는지 안알려줌..다짜고짜 사람을 괴롭히고 죽이는

# MySQL 서버 연결 및 데이터 불러오기

In [113]:
import pandas as pd

In [114]:
conn = pymysql.connect(
    host='127.0.0.1',       # 로컬이면 127.0.0."1 또는 localhost
    user='root',            # MySQL 접속 사용자명
    password='gksmf3191@',    # MySQL 사용자 비번 (없다면 '' 빈문자)
    db='movie',        # 사용할 DB명
    charset='utf8mb4'       # 인코딩(utf8mb4 권장)
)

# 2) 쿼리 실행 후, 결과를 바로 DataFrame으로
query = "SELECT * FROM reviewInformation"
df_added = pd.read_sql(query, conn)

# 3) 연결 닫기
conn.close()

# 4) DataFrame 확인
print(df_added.head())        # 상위 5개 행 출력
print(df_added.info())        # 컬럼 정보

   id movie_title                                            comment  \
0  46          파묘                           사무라이 형님 등장하시기 전까진 재밌었음..   
1  47          파묘                                죽어서도 끊나지 않은 영혼의 한일전   
2  48          파묘                  초중반의 꽤 높은몰입도, 후반으로 갈 수록 산으로 가는 전개   
3  49          파묘                극장에서 떠들고 휴대폰 불빛쏘는 것들을 저 묫자리에 파묻고 싶다   
4  50          파묘  배우들 연기가 진짜 미쳤음!! 특히 김고은 연기... 와 선배 배우들이 왜 칭찬했는...   

         star_score       writing_date released_date          created_at  
0   별점(10점 만점 중)\n7  2024.02.22. 15:25   2024.02.22. 2024-12-25 19:09:59  
1   별점(10점 만점 중)\n8  2024.02.22. 20:53   2024.02.22. 2024-12-25 19:09:59  
2   별점(10점 만점 중)\n6  2024.02.22. 18:27   2024.02.22. 2024-12-25 19:10:00  
3   별점(10점 만점 중)\n6  2024.02.22. 15:20   2024.02.22. 2024-12-25 19:10:01  
4  별점(10점 만점 중)\n10  2024.02.22. 10:28   2024.02.22. 2024-12-25 19:10:01  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 7 columns):
 #   Column  

/opt/homebrew/Caskroom/miniforge/base/envs/AI/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [131]:
df

,id,movie_title,comment,star_score,writing_date,released_date,created_at
0,46,파묘,사무라이 형님 등장하시기 전까진 재밌었음..,별점(10점 만점 중)\n7,2024.02.22. 15:25,2024.02.22.,2024-12-25 19:09:59
1,47,파묘,죽어서도 끊나지 않은 영혼의 한일전,별점(10점 만점 중)\n8,2024.02.22. 20:53,2024.02.22.,2024-12-25 19:09:59
2,48,파묘,"초중반의 꽤 높은몰입도, 후반으로 갈 수록 산으로 가는 전개",별점(10점 만점 중)\n6,2024.02.22. 18:27,2024.02.22.,2024-12-25 19:10:00
3,49,파묘,극장에서 떠들고 휴대폰 불빛쏘는 것들을 저 묫자리에 파묻고 싶다,별점(10점 만점 중)\n6,2024.02.22. 15:20,2024.02.22.,2024-12-25 19:10:01
4,50,파묘,배우들 연기가 진짜 미쳤음!! 특히 김고은 연기... 와 선배 배우들이 왜 칭찬했는...,별점(10점 만점 중)\n10,2024.02.22. 10:28,2024.02.22.,2024-12-25 19:10:01
...,...,...,...,...,...,...,...
445,491,사랑의 하츄핑,딸아이랑 같이 같은데 한 편의 뮤지컬을 보는 느낌이었어요.,별점(10점 만점 중)\n10,2024.08.07. 19:21,2024.08.07.,2024-12-25 19:36:01
446,492,사랑의 하츄핑,"아이랑 같이 봤는데 저도 재미있게 봤습니다.나름 스토리도 짜임새 있고, 영상미도 괜...",별점(10점 만점 중)\n10,2024.08.07. 19:24,2024.08.07.,2024-12-25 19:36:02
447,493,사랑의 하츄핑,사랑의 하츄핑 사랑의 하츄핑 사라애,별점(10점 만점 중)\n10,2024.08.07. 09:02,2024.08.07.,2024-12-25 19:36:02
448,494,사랑의 하츄핑,대사가 완전 오그라들지만 넘사랑스럽다 여아들이보면 빠져들듯 특히 왕자님에게 노래os...,별점(10점 만점 중)\n10,2024.08.09. 17:49,2024.08.07.,2024-12-25 19:36:03


In [132]:
import pandas as pd
import numpy as np

# ------------------------------------------------------------
# 0) 예시: df 라는 Raw DataFrame (가정)
# ------------------------------------------------------------
# df 예시 컬럼: [movie_title, comment, star_score, writing_date, released_date, ...]
# 여기서는 이미 df가 있다고 가정하고, 작업만 보여드립니다.

def process_df(df):
    """
    주어진 df를 요구사항에 맞게 전처리 및 새 테이블 생성한 뒤,
    원본 df는 그대로 유지하고, 가공된 df_return을 반환.
    """

    # -----------------------------
    # 1. comment 컬럼 삭제
    # -----------------------------
    # df를 복사해서 작업 (원본 df 보존을 위해)
    df_processed = df.copy()
    
    if 'comment' in df_processed.columns:
        df_processed = df_processed.drop(columns=['comment'])
    
    # -----------------------------
    # 2. animation 컬럼 생성 (불리언 형태)
    # -----------------------------
    animation_list = ['인사이드 아웃 2', '쿵푸팬더4', '슈퍼배드 4', '위시', '사랑의 하츄핑']
    
    # animation 컬럼: 1(애니메이션) or 0(비애니메이션)
    # isin() 함수를 활용하면 편리
    df_processed['animation'] = df_processed['movie_title'].isin(animation_list).astype(int)

    # -----------------------------
    # 3. writing_date 전처리 (년.월.일 만 남기기)
    #    예) 2024.02.22. 15:25 -> 2024.02.22
    # -----------------------------
    # 1) string slicing or 2) datetime parsing
    # 여기서는 datetime 파싱 후, 다시 yyyy.MM.dd 형식으로 만듦
    def extract_ymd(date_str):
        # 예: '2024.02.22. 15:25'
        # split(' ') 로 쪼개면 ["2024.02.22.", "15:25"]
        # 점이 3개 들어있을 수도 있어 처리 필요
        
        # 간단 방법: 첫 10글자만 slice
        # '2024.02.22' (마지막 점을 제외하거나 포함 여부 주의)
        return date_str.strip()[:10].replace('..','.')
    
    df_processed['writing_date'] = df_processed['writing_date'].apply(extract_ymd)
    
    # 혹은 더 정교하게 datetime으로 파싱해서 재포맷할 수도 있음:
    # df_processed['writing_date'] = pd.to_datetime(df_processed['writing_date'], format='%Y.%m.%d. %H:%M').dt.strftime('%Y.%m.%d')
    # 다만 사용하시는 raw 형태에 따라 format이 다를 수 있어서 주의

    # -----------------------------
    # 4. star_score 전처리
    #    "별점(10점 만점 중)\n7" or "\n10" -> 정수로 (7, 10 등)
    # -----------------------------
    def extract_score(score_str):
        # 예: "별점(10점 만점 중)\n7"
        # 마지막 줄에 "7" 또는 "10"이 들어있음
        # split('\n') 후 마지막 요소를 가져옴
        parts = score_str.split('\n')
        last_part = parts[-1].strip()
        # last_part 가 "10" 혹은 "7" 같은 것이므로 int 변환
        try:
            return int(last_part)
        except ValueError:
            # 만약 예외가 나면 (정수 변환 안 되면) np.nan 반환
            return np.nan
    
    df_processed['star_score'] = df_processed['star_score'].apply(extract_score)

    # ------------------------------------------------------------
    # 새로 만들 테이블: 
    # movie_title, animation, released_date, writing_date,
    # 해당 일자별 리뷰 개수(day_count), 그 일자의 평균(day_avg),
    # 그리고 누적 평균(cum_avg).
    # ------------------------------------------------------------
    # 1) 일단 groupby 로 일자별 평균/개수를 구한다
    group_cols = ['movie_title','writing_date']
    agg_df = df_processed.groupby(group_cols).agg(
        day_count = ('star_score','size'),       # 몇 개가 존재하는지
        day_avg   = ('star_score','mean')        # 해당 일자별 평균
    ).reset_index()

    # 2) 누적 평균(cum_avg): 
    #    같은 movie_title 내에서 writing_date 순으로 누적합 / 누적개수
    #    day_avg * day_count = day_sum
    agg_df['day_sum'] = agg_df['day_avg'] * agg_df['day_count']
    
    # writing_date 를 날짜순 정렬하려면, str -> 날짜 변환 or custom sort
    # 여기서는 str 형태 "2024.02.21" 이 날짜순 정렬 될 수 있도록, datetime 형식으로 변환
    # (단, 실제 date 포맷과 맞춰야 함)
    agg_df['writing_date_dt'] = pd.to_datetime(agg_df['writing_date'], format='%Y.%m.%d', errors='coerce')

    # 정렬
    agg_df = agg_df.sort_values(by=['movie_title','writing_date_dt'])

    # movie_title별로 누적합
    agg_df['cum_sum'] = agg_df.groupby('movie_title')['day_sum'].cumsum()
    agg_df['cum_count'] = agg_df.groupby('movie_title')['day_count'].cumsum()

    agg_df['cum_avg'] = agg_df['cum_sum'] / agg_df['cum_count']

    # 정리: 필요 컬럼만 남기기
    # (writing_date_dt, day_sum) 등은 중간 계산용이므로 나중에 빼도 됨
    agg_df = agg_df.drop(columns=['day_sum','writing_date_dt'])

    # -----------------------------
    # 3) animation, released_date 추가
    # -----------------------------
    # 이 부분이 중요: groupby 결과에는 movie_title, writing_date 만 남아있는데,
    # animation, released_date 는 movie_title 단위로 동일할 것임.
    # => df_processed 에서 movie_title / animation / released_date 만 추출 후 merge
    movie_info = df_processed[['movie_title','animation','released_date']].drop_duplicates()

    # merge
    df_final = pd.merge(agg_df, movie_info, on='movie_title', how='left')

    # 컬럼 순서 정리 (원하는 순서대로)
    # ex) [movie_title, animation, released_date, writing_date, day_count, day_avg, cum_count, cum_avg]
    df_final = df_final[[
        'movie_title','animation','released_date','writing_date',
        'day_count','day_avg','cum_count','cum_avg'
    ]]
    
    # df_processed: 전처리된 raw 형태 (comment삭제, star_score/날짜 정제, animation 컬럼 추가 등)
    # df_final: 요약 테이블
    
    return df_processed, df_final

In [133]:
df_processed, df_new_table = process_df(df)

print("=== 전처리된 Raw Data (comment 삭제, star_score 정제, etc.) ===")
print(df_processed)

print("=== 새로 만든 요약 테이블 ===")
print(df_new_table)

# df 자체는 그대로 (raw df) 남겨둠

=== 전처리된 Raw Data (comment 삭제, star_score 정제, etc.) ===
      id movie_title  star_score writing_date released_date  \
0     46          파묘           7   2024.02.22   2024.02.22.   
1     47          파묘           8   2024.02.22   2024.02.22.   
2     48          파묘           6   2024.02.22   2024.02.22.   
3     49          파묘           6   2024.02.22   2024.02.22.   
4     50          파묘          10   2024.02.22   2024.02.22.   
..   ...         ...         ...          ...           ...   
445  491     사랑의 하츄핑          10   2024.08.07   2024.08.07.   
446  492     사랑의 하츄핑          10   2024.08.07   2024.08.07.   
447  493     사랑의 하츄핑          10   2024.08.07   2024.08.07.   
448  494     사랑의 하츄핑          10   2024.08.09   2024.08.07.   
449  495     사랑의 하츄핑          10   2024.08.10   2024.08.07.   

             created_at  animation  
0   2024-12-25 19:09:59          0  
1   2024-12-25 19:09:59          0  
2   2024-12-25 19:10:00          0  
3   2024-12-25 19:10:01          0  
4 

In [134]:
import pandas as pd
import os
import xlwt

# 두 개의 예시 DataFrame (이미 생성된 상태라고 가정)
# df_processed, df_new_table

# 저장 경로 지정 (macOS 기준 예시)
save_path = "/Users/parkhaneul/Downloads"

# 1) df_processed → CSV 저장
processed_filename = "df_processed.csv"
processed_fullpath = os.path.join(save_path, processed_filename)

df_processed.to_csv(processed_fullpath, index=False, encoding='utf-8-sig')
print(f"df_processed를 CSV로 저장했습니다: {processed_fullpath}")

# 2) df_new_table → CSV 저장
new_table_filename = "df_new_table.csv"
new_table_fullpath = os.path.join(save_path, new_table_filename)

df_new_table.to_csv(new_table_fullpath, index=False, encoding='utf-8-sig')
print(f"df_new_table을 CSV로 저장했습니다: {new_table_fullpath}")


df_processed를 CSV로 저장했습니다: /Users/parkhaneul/Downloads/df_processed.csv
df_new_table을 CSV로 저장했습니다: /Users/parkhaneul/Downloads/df_new_table.csv


In [136]:
csv_path = "/Users/parkhaneul/Downloads/df_new_table.csv"
df = pd.read_csv(csv_path)

# 2) 각 영화별로 총 며칠간 데이터가 있는지 (writing_date 기준)
#    -> movie_title 그룹별로 writing_date 고유 개수(nunique)
days_per_movie = df.groupby('movie_title')['writing_date'].nunique()

# 3) 결과 출력
print("영화별로 총 몇 일치의 데이터가 있는지:")
for movie, num_days in days_per_movie.items():
    print(f"{movie} : {num_days}일")

영화별로 총 몇 일치의 데이터가 있는지:
범죄도시4 : 5일
베테랑2 : 2일
사랑의 하츄핑 : 7일
슈퍼배드 4 : 9일
웡카 : 6일
위시 : 6일
인사이드 아웃 2 : 7일
쿵푸팬더4 : 9일
파묘 : 5일
파일럿 : 11일


# 각 영화별 박스오피스 데이터

In [183]:
import os
import pandas as pd

# 기본 경로와 영화 목록 정의
base_path = '/Users/parkhaneul/Downloads/'
movies = [
    "범죄도시4", "베테랑2", "사랑의 하츄핑", "슈퍼배드 4", 
    "웡카", "위시", "인사이드 아웃 2", "쿵푸팬더4", 
    "파묘", "파일럿"
]

# 모든 영화 파일 존재 여부 확인
missing_files = [movie for movie in movies if not os.path.exists(f"{base_path}{movie}.csv")]

if missing_files:
    print(f"누락된 파일: {missing_files}")
else:
    # 개봉일 정보 로드
    df_new_table_path = f"{base_path}df_new_table.csv"
    if not os.path.exists(df_new_table_path):
        print("df_new_table.csv 파일이 없습니다.")
    else:
        # 개봉일 테이블 로드
        df_new_table = pd.read_csv(df_new_table_path, encoding='utf-8-sig')

        # 열 이름을 소문자로 변환하여 키 오류 방지
        df_new_table.columns = df_new_table.columns.str.strip().str.lower()

        # 처리된 데이터 저장을 위한 리스트
        processed_dfs = []

        # 각 영화 처리
        for movie in movies:
            movie_path = f"{base_path}{movie}.csv"
            try:
                # .csv 파일 읽기 (헤더는 A4부터 시작, 즉 3번째 행부터 시작)
                df_movie = pd.read_csv(movie_path, encoding='utf-8-sig', engine='python', header=3)
            except UnicodeDecodeError:
                try:
                    df_movie = pd.read_csv(movie_path, encoding='latin1', engine='python', header=3)
                except Exception as e:
                    print(f"파일 읽기 오류 {movie_path}: {e}")
                    continue

            # 열 이름을 소문자로 변환하여 키 오류 방지
            df_movie.columns = df_movie.columns.str.strip().str.lower()

            # 개봉일 및 애니메이션 여부 가져오기
            release_date_row = df_new_table.loc[df_new_table['movie_title'] == movie]

            if release_date_row.empty:
                print(f"개봉일 또는 애니메이션 여부를 찾을 수 없습니다: {movie}")
                continue

            release_date = release_date_row['released_date'].values[0]
            animation_status = release_date_row['animation'].values[0]

            # 날짜 변환
            df_movie['날짜'] = pd.to_datetime(df_movie['날짜'], errors='coerce')
            release_date = pd.to_datetime(release_date, errors='coerce')

            if pd.isna(release_date):
                print(f"유효하지 않은 개봉일: {movie}")
                continue

            # 개봉일로부터 11일간의 데이터 필터링
            df_filtered = df_movie[(df_movie['날짜'] >= release_date) &
                                   (df_movie['날짜'] < release_date + pd.Timedelta(days=11))]

            if df_filtered.empty:
                print(f"개봉 기간 동안 데이터를 찾을 수 없습니다: {movie}")
                continue

            # 애니메이션 여부 열 추가
            df_filtered['animation'] = animation_status

            # 영화명 열 추가
            df_filtered['movie_title'] = movie

            # 처리된 데이터프레임 추가
            processed_dfs.append(df_filtered)

        # 처리된 데이터프레임이 있는지 확인
        if processed_dfs:
            # 모든 데이터프레임 결합
            final_df = pd.concat(processed_dfs, ignore_index=True)

            # 최종 데이터프레임을 CSV 파일로 저장
            final_csv_path = f"{base_path}final_movies_data.csv"
            final_df.to_csv(final_csv_path, index=False, encoding='utf-8-sig')
            print(f"최종 CSV가 생성되었습니다: {final_csv_path}")
        else:
            print("결합할 유효한 데이터가 없습니다.")


최종 CSV가 생성되었습니다: /Users/parkhaneul/Downloads/final_movies_data.csv


/var/folders/4n/9zzwhzvd4gq2yltkpsdf6ph00000gn/T/ipykernel_57620/2009763108.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['animation'] = animation_status
/var/folders/4n/9zzwhzvd4gq2yltkpsdf6ph00000gn/T/ipykernel_57620/2009763108.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['movie_title'] = movie
/var/folders/4n/9zzwhzvd4gq2yltkpsdf6ph00000gn/T/ipykernel_57620/2009763108.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


# 최최종 데이터프레임 만들기 

In [186]:
import os
import pandas as pd

# 기본 경로와 파일 경로 정의
base_path = '/Users/parkhaneul/Downloads/'
final_data_path = f"{base_path}final_movies_data.csv"
df_new_table_path = f"{base_path}df_new_table.csv"

# 필요한 데이터 컬럼 정의
define_columns = ["movie_title", "day", "adopters_t", "cumulative_adopters_t", 
                  "review_count", "review_avg_score", "animation"]

# 파일 존재 여부 확인
if not os.path.exists(final_data_path) or not os.path.exists(df_new_table_path):
    print("필요한 파일이 누락되었습니다.")
else:
    # 데이터 로드
    final_df = pd.read_csv(final_data_path, encoding='utf-8-sig')
    df_new_table = pd.read_csv(df_new_table_path, encoding='utf-8-sig')

    # 열 이름을 소문자로 통일
    final_df.columns = final_df.columns.str.strip().str.lower()
    df_new_table.columns = df_new_table.columns.str.strip().str.lower()

    # 날짜 포맷 변환
    final_df['날짜'] = pd.to_datetime(final_df['날짜'], errors='coerce')
    df_new_table['writing_date'] = pd.to_datetime(df_new_table['writing_date'], errors='coerce')

    # 최종 데이터 저장 리스트
    processed_rows = []

    # 영화별 데이터 처리
    for movie in final_df['movie_title'].unique():
        movie_data = final_df[final_df['movie_title'] == movie].copy()
        movie_info = df_new_table[df_new_table['movie_title'] == movie]

        # 'day' 및 다른 데이터 생성
        for idx, row in movie_data.iterrows():
            day = idx % 11  # day는 0~10까지 반복
            adopters_t = row['관객수']
            cumulative_adopters_t = row['누적관객수']

            # df_new_table에서 'writing_date'와 '날짜' 일치하는 데이터 가져오기
            matching_info = movie_info[movie_info['writing_date'] == row['날짜']]

            if not matching_info.empty:
                review_count = matching_info['cum_count'].values[0]
                review_avg_score = matching_info['cum_avg'].values[0]
            else:
                review_count = None
                review_avg_score = None

            animation = row['animation']

            # 최종 행 데이터 추가
            processed_rows.append({
                "movie_title": movie,
                "day": day,
                "adopters_t": adopters_t,
                "cumulative_adopters_t": cumulative_adopters_t,
                "review_count": review_count,
                "review_avg_score": review_avg_score,
                "animation": animation
            })

    # 최종 데이터프레임 생성
    final_processed_df = pd.DataFrame(processed_rows, columns=define_columns)

    # 결과 저장
    final_output_path = f"{base_path}processed_movies_data.csv"
    final_processed_df.to_csv(final_output_path, index=False, encoding='utf-8-sig')
    print(f"최종 데이터 CSV가 생성되었습니다: {final_output_path}")


최종 데이터 CSV가 생성되었습니다: /Users/parkhaneul/Downloads/processed_movies_data.csv
